In [1]:
from query.datasets.prelude import *
from timeit import default_timer as now

q1 = FaceGender.objects.values('id')
q2 = FaceGender.objects.values('id', 'face__person__frame__video__show')    

In [2]:
def profile(s, f):
    start = now()
    f()
    print('{}: {:.3f}'.format(s, now() - start))
    
profile('nojoin', lambda: list(q1))
profile('withjoin', lambda: list(q2))

nojoin: 1.992
withjoin: 14.583


In [3]:
q1.print_sql()
print('')
q1.explain()

SELECT "query_tvnews_facegender"."id"
FROM "query_tvnews_facegender"

Seq Scan on query_tvnews_facegender  (cost=0.00..15164.10 rows=984310 width=4) (actual time=0.011..125.764 rows=984310 loops=1)
Planning time: 0.055 ms
Execution time: 181.377 ms


In [4]:
q2.print_sql()
print('')
q2.explain()

SELECT "query_tvnews_facegender"."id",
       "query_tvnews_video"."show_id"
FROM "query_tvnews_facegender"
INNER JOIN "query_tvnews_face" ON ("query_tvnews_facegender"."face_id" = "query_tvnews_face"."id")
INNER JOIN "query_tvnews_person" ON ("query_tvnews_face"."person_id" = "query_tvnews_person"."id")
INNER JOIN "query_tvnews_frame" ON ("query_tvnews_person"."frame_id" = "query_tvnews_frame"."id")
INNER JOIN "query_tvnews_video" ON ("query_tvnews_frame"."video_id" = "query_tvnews_video"."id")

Hash Join  (cost=67467.34..918389.29 rows=984310 width=8) (actual time=3122.699..11024.387 rows=984310 loops=1)
  Hash Cond: (query_tvnews_frame.video_id = query_tvnews_video.id)
  ->  Nested Loop  (cost=67453.39..905585.52 rows=984310 width=8) (actual time=3122.536..10740.431 rows=984310 loops=1)
        ->  Hash Join  (cost=67452.95..367599.54 rows=984310 width=8) (actual time=3122.514..5782.762 rows=984310 loops=1)
              Hash Cond: (query_tvnews_face.id = query_tvnews_facegender.fac